### 1. Read data (SST, Soil Moisture). This includes temporal subsetting.

In [1]:
from cate.core.ds import DATA_STORE_REGISTRY
from cate.util import ConsoleMonitor
import cate.ops as ops

monitor = ConsoleMonitor()
data_store = DATA_STORE_REGISTRY.get_data_store('esa_cci_odp')

In [2]:
data_store.query('sst')

[esacci.GHRSST.second.L3U.SSTskin.AATSR.Envisat.AATSR.1-0.r1,
 esacci.GHRSST.second.L3U.SSTskin.AATSR.Envisat.AATSR.1-1.r1,
 esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.1-1.r1,
 esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.1-0.r1,
 esacci.GHRSST.second.L2P.SSTskin.AVHRR.NOAA-12.AVHRR12_G.1-0.r1]

In [3]:
data_store.query('soil')

[esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.COMBINED.02-2.r1,
 esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.PASSIVE.02-2.r1,
 esacci.SOILMOISTURE.day.L3S.SSMS.multi-sensor.multi-platform.ACTIVE.02-2.r1,
 esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.COMBINED.02-1.r1,
 esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.PASSIVE.02-1.r1,
 esacci.SOILMOISTURE.day.L3S.SSMS.multi-sensor.multi-platform.ACTIVE.02-1.r1]

In [4]:
sm = ops.open_dataset('esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.COMBINED.02-2.r1',
                       '2000-01-01',
                       '2000-01-31', sync=True, monitor=monitor)

In [5]:
sst = ops.open_dataset('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.1-1.r1',
                       '2000-01-01',
                       '2000-01-31', sync=True, monitor=monitor)

In [ ]:
sst

In [ ]:
sst = sst.chunk(chunks={'lat':36, 'lon':72, 'time':31})

In [ ]:
sm

In [6]:
sm = sm.chunk(chunks={'lat':72, 'lon':144, 'time':31})

In [16]:
def _add_time(da, time):
    coords = {'lat':da.lat, 'lon':da.lon, 'time':time}
    new_da = xr.DataArray(da.data, coords=coords, dims=['lat', 'lon'])
    return xr.concat([new_da], 'time')
    

In [ ]:
list(sst.groupby('time.month'))

In [ ]:
sst.nbytes / (2 ** 30)

In [ ]:
sm_mean = sm.groupby('time.month').mean('time', skipna=True)

In [9]:
sm_mean.sm.lat

<xarray.DataArray 'lat' (lat: 720)>
dask.array<xarray-..., shape=(720,), dtype=float32, chunksize=(72,)>
Coordinates:
  * lat      (lat) float32 89.875 89.625 89.375 89.125 88.875 88.625 88.375 ...
Attributes:
    standard_name: latitude
    units: degrees_north
    valid_range: [-90.  90.]
    _CoordinateAxisType: Lat

In [7]:
sm_mean = sm.mean('time', keep_attrs=True, skipna=True)

In [25]:
t_dim = xr.DataArray([datetime(2000,1,1)], name='time')

In [27]:
t_dim[0]

<xarray.DataArray 'time' ()>
numpy.datetime64('2000-01-01T01:00:00.000000000+0100')
Coordinates:
    dim_0    int64 0

In [29]:
sm_mean_time = xr.concat([sm_mean], 'time')

In [30]:
sm_mean_time.time[0] = datetime(2000,1,1)

TypeError: Coordinate values cannot be modified

In [15]:
import xarray as xr
from datetime import datetime
kwargs = {'time':[datetime(2000,1,1)]}
sm_mean_time = sm_mean.apply(_add_time, **kwargs)

ValueError: coordinate time has dimensions ('time',), but these are not a subset of the DataArray dimensions ['lat', 'lon']

In [8]:
ops.save_dataset(sm_mean, '/home/ccitbx/Desktop/sm_mean.nc')

/home/ccitbx/miniconda3/envs/ect_env/lib/python3.5/site-packages/dask/array/numpy_compat.py:44: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [ ]:
%matplotlib inline
sm_mean.sm.plot()

In [ ]:
sst_mean = sst.mean('time', skipna=True)

In [ ]:
ops.save_dataset(sst_mean, '/home/ccitbx/Desktop/sst_mean.nc')

In [ ]:
sm

In [ ]:
sm_rech = sm.chunk(chunks={'time':60, 'lat': 72, 'lon': 144})

In [ ]:
sm_rech.sm.data

In [ ]:
sm_monthly = sm_rech.groupby('time.month').mean('time')

In [ ]:
sm_res = sm_rech.resample('M','time',skipna=True)

In [ ]:
sm_res

In [ ]:
sst.load()

In [ ]:
sst_rech = sst.chunk(chunks = {'lat':360, 'lon':720, 'time':60})

In [ ]:
sst_res = sst_rech.resample('M', 'time', skipna=True)

In [ ]:
sst_res

### 2. Select required geophysical quantities (sea surface temperature, surface soil moisture volumetric absolutes), this should keep associated uncertainties

In [ ]:
sm = ops.select_var(sm, 'sm')
sst = ops.select_var(sst, 'analysed_sst')

In [ ]:
sst

In [ ]:
sst_rech = sst.chunk(chunks={'lat':400, 'lon':400, 'time':60})

In [ ]:
sst_rech.resample('M','time',skipna=True)

In [ ]:
sm_mean = sm.groupby('time').mean()

In [ ]:
sm_mean.sm

In [ ]:
sst_mean = sm.groupby('time').mean()

In [ ]:
sst_mean

In [ ]:
%matplotlib inline

In [ ]:
#ops.plot_map(sm, var='sm', time=2)

In [ ]:
#ops.plot_map(sst, var='analysed_sst', time=0)

### 3. Temporal aggregation

In [ ]:
sst_monthly = sst.resample('M','time', skipna=True)

In [ ]:
sst_monthly

In [ ]:
sm_monthly = sm.resample('M', 'time', skipna=True)

In [ ]:
ops.plot_map(sm_monthly, var='sm', time=1)

In [ ]:
sst

In [ ]:
sst_nt = ops.select_var(sst, 'a*,l*,m*,s*')

In [ ]:
sst_nt

In [ ]:
sst/4

### 3. Co-register data sets (spline, propagate uncertainties)

### 4. Calculate N3.4 index

### 5. Log transformation of soil moisture data

### 6. absolute anomaly of log transformed SM data with respect to mean of reference period

### 7. Pearson correlation with 30 days lag time (between ENSO index and soil moisture)

### 8. Select a point in South-East Asia

### 9. Pearson correlation with 30 days lag time